In [1]:

import numpy as np
from keras.backend import clear_session
from keras.callbacks import TensorBoard
from keras.layers import Activation, Dropout
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.models import Sequential
from time import strftime
from keras.optimizers import Adam, SGD
from keras_tqdm import TQDMNotebookCallback


Using TensorFlow backend.


In [13]:
DATASET = "cat-dog-25000-GrayScale-(50, 50).npy"
LR = 0.001
ROW_SIZE = 50
COL_SIZE = 50
EPOCHS = 10
CLASSES = 2 #[ DOG, CAT ]
MODEL_NAME = f"cat-dog-{EPOCHS}-4"

In [3]:
def lprNet():
    new_model = Sequential()

    new_model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(ROW_SIZE, COL_SIZE, 1)))
    new_model.add(Activation('relu'))
    new_model.add(MaxPooling2D(pool_size=(2,2)))

    new_model.add(Conv2D(64, kernel_size=(3, 3)))
    new_model.add(Activation('relu'))
    new_model.add(MaxPooling2D(pool_size=(2,2)))

    new_model.add(Conv2D(32, kernel_size=(3, 3)))
    new_model.add(Activation('relu'))
    new_model.add(MaxPooling2D(pool_size=(2,2)))

    new_model.add(Conv2D(32, kernel_size=(3, 3)))
    new_model.add(Activation('relu'))
    new_model.add(MaxPooling2D(pool_size=(2,2)))

    new_model.add(Flatten())
    new_model.add(Dense(64))
    new_model.add(Activation('relu'))
    new_model.add(Dropout(0.2))

    new_model.add(Dense(2, activation="softmax"))

    return new_model
    

In [10]:
def generateData():
    data = np.load(DATASET, allow_pickle=True)
    np.random.shuffle(data)
    
    X = []
    y = []
    
    def to_categorical(y):
        if y.upper() == 'CAT':
            return [0,1]
        if y.upper() == 'DOG':
             return [1,0]
    
    for img in data:
        X.append(img[0])
        y.append(to_categorical(img[1]))
    X = np.array(X).reshape(-1, ROW_SIZE, COL_SIZE, 1)
    return X, np.array(y)


In [11]:
X, y = generateData()
X = X/255.0

modelo = lprNet()
optimizer = SGD(momentum=0.01, nesterov=True)
tensorBoard = TensorBoard(log_dir=f"../logs/alexandre/{MODEL_NAME}-{strftime('%d-%m-%Y')}-{strftime('%H-%M-%S')}")

modelo.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
modelo.fit(X,
           y,
           batch_size=32,
           epochs=EPOCHS,
           validation_split=0.1,
           callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True),
                      tensorBoard],
           verbose=0
            )
modelo.save(f"{MODEL_NAME}.h5")
clear_session()
